# matching clusters

In [7]:
import json
import pandas as pd

In [3]:
with open('../data/output/document_table.json', 'rb') as f:
    doc_table = json.load(f)

In [5]:
doc_table.keys()

dict_keys(['lda_km_topic_map', 'km_lda_topic_map', 'lda_dec_topic_map', 'dec_lda_topic_map', 'rows'])

In [9]:
doc_df = pd.DataFrame(doc_table['rows'])

In [10]:
doc_df.head()

,document,topic_dec,topic_km,topic_lda
0,Analysis of efficacy,0,3,0
1,Comparisons of postoperative CA19-9 levels on ...,4,0,7
2,Pattern of disease relapse,3,4,1
3,Grade 1–5 adverse events with gemcitabine alon...,2,1,1
4,Treatment with zoledronic acid,2,0,6


In [17]:
doc_table['dec_lda_topic_map']

{'0': [0],
 '1': [4],
 '2': [1],
 '3': [0],
 '4': [7],
 '5': [2],
 '6': [7],
 '7': [3]}

In [27]:
dec_doc_count = doc_df.groupby('topic_dec').document.count().to_dict()

In [37]:
dec_lda_df = doc_df.groupby(['topic_dec', 'topic_lda']).count().reset_index()
dec_lda_df['doc_ratio'] = dec_lda_df.apply(lambda x: x.document / dec_doc_count[x.topic_dec], axis=1)

In [48]:
dec_lda_df['max_doc_ratio'] = dec_lda_df.groupby('topic_dec')['doc_ratio'].transform('max')

In [50]:
dec_lda_df[dec_lda_df.doc_ratio == dec_lda_df.max_doc_ratio]

,topic_dec,topic_lda,document,topic_km,doc_ratio,max_doc_ratio
0,0,0,2553,2553,0.728804,0.728804
12,1,4,1892,1892,0.555164,0.555164
17,2,1,1827,1827,0.495256,0.495256
24,3,0,867,867,0.263046,0.263046
39,4,7,931,931,0.256262,0.256262
42,5,2,1281,1281,0.391983,0.391983
55,6,7,1304,1304,0.347178,0.347178
59,7,3,1727,1727,0.506898,0.506898


In [72]:
dec_lda_df['doc_ratio_rank'] = dec_lda_df.groupby('topic_dec').doc_ratio.rank(ascending=False)

In [73]:
dec_lda_df.head()

,topic_dec,topic_lda,document,topic_km,doc_ratio,max_doc_ratio,doc_ratio_rank
0,0,0,2553,2553,0.728804,0.728804,1.0
1,0,1,87,87,0.024836,0.728804,8.0
2,0,2,125,125,0.035684,0.728804,5.0
3,0,3,88,88,0.025121,0.728804,7.0
4,0,4,151,151,0.043106,0.728804,4.0


In [105]:
dec_lda_matching = {}

In [111]:
dec_lda_df[dec_lda_df.doc_ratio_rank == 5].sort_values('doc_ratio', ascending=False)

,topic_dec,topic_lda,document,topic_km,doc_ratio,max_doc_ratio,doc_ratio_rank
33,4,1,400,400,0.110102,0.256262,5.0
29,3,5,362,362,0.109830,0.263046,5.0
46,5,6,337,337,0.103121,0.391983,5.0
49,6,1,327,327,0.087061,0.347178,5.0
8,1,0,194,194,0.056925,0.555164,5.0
20,2,4,178,178,0.048252,0.495256,5.0
63,7,7,147,147,0.043146,0.506898,5.0
2,0,2,125,125,0.035684,0.728804,5.0


In [106]:
for i in range(1, 9, 1):
    for idx, row in dec_lda_df[dec_lda_df.doc_ratio_rank == i].sort_values('doc_ratio', ascending=False).iterrows():
        if row.topic_lda not in dec_lda_matching.values() and row.topic_dec not in dec_lda_matching.keys():
            dec_lda_matching[row.topic_dec] = row.topic_lda
    if len(dec_lda_matching) == 8:
        break;

In [107]:
for k in dec_lda_matching.keys():
    print(k, dec_lda_matching[k])

0.0 0.0
1.0 4.0
7.0 3.0
2.0 1.0
5.0 2.0
6.0 7.0
4.0 6.0
3.0 5.0


In [94]:
len(dec_lda_matching)

7

In [ ]:
0.0 0.0
1.0 4.0
7.0 3.0
2.0 1.0
5.0 2.0
6.0 7.0

In [63]:
matching = []
for ind, grp in dec_lda_df[dec_lda_df.doc_ratio == dec_lda_df.max_doc_ratio].groupby('topic_lda'):#.agg({'topic_dec': 'unique'})
    print(grp)
    print(grp[grp.doc_ratio == grp.doc_ratio.max()].topic_dec)
    matching.append({'dec': grp[grp.doc_ratio == grp.doc_ratio.max()].topic_dec.values[0], 'lda':ind})
    print(matching)

    topic_dec  topic_lda  document  topic_km  doc_ratio  max_doc_ratio
0           0          0      2553      2553   0.728804       0.728804
24          3          0       867       867   0.263046       0.263046
0    0
Name: topic_dec, dtype: int64
[{'dec': 0, 'lda': 0}]
    topic_dec  topic_lda  document  topic_km  doc_ratio  max_doc_ratio
17          2          1      1827      1827   0.495256       0.495256
17    2
Name: topic_dec, dtype: int64
[{'dec': 0, 'lda': 0}, {'dec': 2, 'lda': 1}]
    topic_dec  topic_lda  document  topic_km  doc_ratio  max_doc_ratio
42          5          2      1281      1281   0.391983       0.391983
42    5
Name: topic_dec, dtype: int64
[{'dec': 0, 'lda': 0}, {'dec': 2, 'lda': 1}, {'dec': 5, 'lda': 2}]
    topic_dec  topic_lda  document  topic_km  doc_ratio  max_doc_ratio
59          7          3      1727      1727   0.506898       0.506898
59    7
Name: topic_dec, dtype: int64
[{'dec': 0, 'lda': 0}, {'dec': 2, 'lda': 1}, {'dec': 5, 'lda': 2}, {'dec': 

In [64]:
len(matching)

6

In [67]:
# 3, 4
# 5, 6
dec_lda_df[dec_lda_df.topic_dec==4]

,topic_dec,topic_lda,document,topic_km,doc_ratio,max_doc_ratio
32,4,0,156,156,0.042940,0.256262
33,4,1,400,400,0.110102,0.256262
34,4,2,484,484,0.133223,0.256262
35,4,3,243,243,0.066887,0.256262
36,4,4,239,239,0.065786,0.256262
37,4,5,580,580,0.159648,0.256262
38,4,6,600,600,0.165153,0.256262
39,4,7,931,931,0.256262,0.256262
